In [2]:
import rawpy
import cv2
from PIL import Image
import numpy as np
from multiprocessing.pool import Pool
import utils
import os


###############################################################################
# Instantiation
###############################################################################
lum_downsample = utils.Downsampling(ratio='4:4:4')
chr_downsample = utils.Downsampling(ratio='4:2:0')
image_block = utils.ImageBlock(block_height=8, block_width=8)
dct2d = utils.DCT2D(norm='ortho')
quantization = utils.Quantization()
zigzagScanning = utils.ZigzagScanning()
rle = utils.RLE()
entropy = utils.Entropy()
###############################################################################
# Preprocess
###############################################################################
# Read raw image file as array
# Generate random pixel values
raw = rawpy.imread(os.path.join('images', 'Nikon-D3500-Shotkit-4.NEF'))

    




In [3]:
rgb_img = raw.postprocess()


In [4]:
rgb_img.shape

(4016, 6016, 3)

In [5]:
# Colorspace transform (RGB -> YCrCb)
ycc_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2YCrCb)


In [6]:
ycc_img

array([[[161, 146, 129],
        [161, 148, 129],
        [163, 147, 127],
        ...,
        [101, 138, 130],
        [101, 142, 130],
        [ 99, 143, 132]],

       [[161, 145, 129],
        [162, 144, 128],
        [163, 142, 126],
        ...,
        [ 99, 137, 131],
        [ 99, 137, 131],
        [100, 137, 131]],

       [[163, 143, 127],
        [164, 139, 126],
        [161, 139, 127],
        ...,
        [ 97, 134, 133],
        [ 99, 132, 131],
        [101, 129, 129]],

       ...,

       [[ 91, 139, 121],
        [ 92, 141, 120],
        [ 92, 142, 120],
        ...,
        [ 86, 147, 119],
        [ 86, 149, 119],
        [ 89, 146, 116]],

       [[ 89, 136, 122],
        [ 91, 139, 121],
        [ 92, 143, 121],
        ...,
        [ 88, 146, 116],
        [ 87, 145, 117],
        [ 87, 145, 118]],

       [[ 88, 137, 123],
        [ 90, 140, 122],
        [ 92, 143, 122],
        ...,
        [ 88, 146, 114],
        [ 88, 144, 116],
        [ 87, 144, 118]]

In [7]:
# Center
ycc_img = ycc_img.astype(int)-128

In [8]:
ycc_img

array([[[ 33,  18,   1],
        [ 33,  20,   1],
        [ 35,  19,  -1],
        ...,
        [-27,  10,   2],
        [-27,  14,   2],
        [-29,  15,   4]],

       [[ 33,  17,   1],
        [ 34,  16,   0],
        [ 35,  14,  -2],
        ...,
        [-29,   9,   3],
        [-29,   9,   3],
        [-28,   9,   3]],

       [[ 35,  15,  -1],
        [ 36,  11,  -2],
        [ 33,  11,  -1],
        ...,
        [-31,   6,   5],
        [-29,   4,   3],
        [-27,   1,   1]],

       ...,

       [[-37,  11,  -7],
        [-36,  13,  -8],
        [-36,  14,  -8],
        ...,
        [-42,  19,  -9],
        [-42,  21,  -9],
        [-39,  18, -12]],

       [[-39,   8,  -6],
        [-37,  11,  -7],
        [-36,  15,  -7],
        ...,
        [-40,  18, -12],
        [-41,  17, -11],
        [-41,  17, -10]],

       [[-40,   9,  -5],
        [-38,  12,  -6],
        [-36,  15,  -6],
        ...,
        [-40,  18, -14],
        [-40,  16, -12],
        [-41,  16, -10]]

In [9]:
# Downsampling
Y = lum_downsample(ycc_img[:,:,0])
Cr = chr_downsample(ycc_img[:,:,1])
Cb = chr_downsample(ycc_img[:,:,2])
ycc_img = np.stack((Y, Cr, Cb), axis=2)

In [10]:
Y

array([[ 33,  33,  35, ..., -27, -27, -29],
       [ 33,  34,  35, ..., -29, -29, -28],
       [ 35,  36,  33, ..., -31, -29, -27],
       ...,
       [-37, -36, -36, ..., -42, -42, -39],
       [-39, -37, -36, ..., -40, -41, -41],
       [-40, -38, -36, ..., -40, -40, -41]])

In [11]:
Y[0].size

6016

In [12]:
Cr[0].size

6016

In [13]:
Cb.size

24160256

In [14]:
ycc_img

array([[[ 33,  18,   1],
        [ 33,  18,   1],
        [ 35,  17,  -1],
        ...,
        [-27,   9,   3],
        [-27,  12,   3],
        [-29,  12,   3]],

       [[ 33,  18,   1],
        [ 34,  18,   1],
        [ 35,  17,  -1],
        ...,
        [-29,   9,   3],
        [-29,  12,   3],
        [-28,  12,   3]],

       [[ 35,  13,  -2],
        [ 36,  13,  -2],
        [ 33,  13,  -1],
        ...,
        [-31,   7,   5],
        [-29,   4,   2],
        [-27,   4,   2]],

       ...,

       [[-37,  14,  -7],
        [-36,  14,  -7],
        [-36,  13,  -8],
        ...,
        [-42,  18,  -9],
        [-42,  18, -11],
        [-39,  18, -11]],

       [[-39,  10,  -6],
        [-37,  10,  -6],
        [-36,  13,  -6],
        ...,
        [-40,  19, -12],
        [-41,  16, -11],
        [-41,  16, -11]],

       [[-40,  10,  -6],
        [-38,  10,  -6],
        [-36,  13,  -6],
        ...,
        [-40,  19, -12],
        [-40,  16, -11],
        [-41,  16, -11]]

In [15]:
# Create 8x8 blocks
blocks, indices = image_block.forward(ycc_img)

In [125]:
(print(np.where(x==9)) for x in range(0,13))

<generator object <genexpr> at 0x000001F3BCA6CE50>

In [127]:
for index in indices:
    if index[0]!=0:
        print(np.where(indices == index))
        break

(array([      0,       0,       1, ..., 1132503, 1132506, 1132509],
      dtype=int64), array([1, 2, 1, ..., 2, 2, 2], dtype=int64))


In [140]:

def predict_block(block, index,blocks,count):
    prediction_error_array =[]
    count_vertical = 0
    count_horizontal = 0
    for j in range(0,len(index)):
        
        blocks_transpose = np.transpose(blocks)
        if index[0]==0 and index[1]!=0:
            # Select previous block vertically
            
            prev_block = (blocks[count-1][index[2]] )
            last_column = np.transpose(prev_block)[7]
            prediction  = np.mean(last_column)*np.ones([8,8],'float32')
            prediction_error = blocks[index][index[2]]-prediction
            prediction_error_array.append(prediction_error)
            count_vertical+=1
        
        elif index[1]==0 and index[0]!=0:
            # Select previous block horizontally
            print(index[1],index[0])
            prev_block = (blocks_transpose[count-1][index[2]] )
            print(prev_block)
            last_column = prev_block[7]
            prediction  = np.mean(last_column)*np.ones([8,8],'float32')
            prediction_error = blocks[index][index[2]]-prediction
            count_horizontal+=1
        elif index[1]==0 and index[0]!=0:
            # Select previous block horizontally
            print(index[1],index[0])
            prev_block = (blocks_transpose[count-1][index[2]] )
            print(prev_block)
            last_column = prev_block[7]
            prediction  = np.mean(last_column)*np.ones([8,8],'float32')
            prediction_error = blocks[index][index[2]]-prediction
            count_horizontal+=1
    return prediction_error_array

In [143]:
count = 0 
print(predict_block(blocks[2250],indices[2250],blocks,2250))

[array([[31., 31., 33., 32., 33., 31., 30., 28.],
       [31., 32., 33., 32., 33., 32., 31., 31.],
       [33., 34., 31., 30., 32., 35., 32., 32.],
       [31., 32., 32., 32., 30., 32., 30., 31.],
       [30., 33., 33., 34., 32., 31., 31., 29.],
       [29., 32., 32., 32., 30., 35., 33., 32.],
       [30., 30., 30., 32., 32., 35., 33., 34.],
       [31., 30., 28., 31., 31., 31., 32., 29.]]), array([[31., 31., 33., 32., 33., 31., 30., 28.],
       [31., 32., 33., 32., 33., 32., 31., 31.],
       [33., 34., 31., 30., 32., 35., 32., 32.],
       [31., 32., 32., 32., 30., 32., 30., 31.],
       [30., 33., 33., 34., 32., 31., 31., 29.],
       [29., 32., 32., 32., 30., 35., 33., 32.],
       [30., 30., 30., 32., 32., 35., 33., 34.],
       [31., 30., 28., 31., 31., 31., 32., 29.]]), array([[31., 31., 33., 32., 33., 31., 30., 28.],
       [31., 32., 33., 32., 33., 32., 31., 31.],
       [33., 34., 31., 30., 32., 35., 32., 32.],
       [31., 32., 32., 32., 30., 32., 30., 31.],
       [30., 33

In [88]:
QP=15

# # Vertical Prediction 
# def predictor(prevBlock, currBlock):

# Horizontal Prediction
reconstsructed_blocks =[]
def predict_block(block, index,blocks):
    
    # detect Borders
    if index[0]==0 and index[1] == 0:
        # DCT
        encoded = dct2d.forward(block)
        if index[2] == 0:
            channel_type = 'lum'
        else:
            channel_type = 'chr'
            
        # Quantization
        encoded = quantization.forward(encoded, channel_type,QP)
        
        # RLE + zigzag scanning
        encoded = zigzagScanning.forward(encoded)
        encoded = rle.forward(encoded)
        
        decoded = rle.backward(encoded)
        # Reverse RLE + zigzag scanning
        decoded = zigzagScanning.backward(decoded)
        # Dequantization
        decoded = quantization.backward(decoded, channel_type,QP)
        
        # Reverse DCT
        reconstructed_block = dct2d.backward(decoded)
        
        reconstsructed_blocks.append(reconstructed_block)
        
    # First row vertical prediction 
    
    elif index[0]==0:
        # Select previous block vertically
        
        prev_block = blocks[index[0]][index[1]-8][index[2]]
        mean_last_column = np.mean(np.transpose(prev_block)[7])
    
def process_block(block, index, toEncode_Y, toEncode_CbCr):

    #Prediction  -> Prediction error
    
    # DCT
    encoded = dct2d.forward(block)
    if index[2] == 0:
        channel_type = 'lum'
    else:
        channel_type = 'chr'
        
    # Quantization
    encoded = quantization.forward(encoded, channel_type,QP)
    
    # RLE + zigzag scanning
    encoded = zigzagScanning.forward(encoded)
    encoded = rle.forward(encoded)
    
    assert channel_type in ('lum', 'chr')
        
    if channel_type == 'lum':
        toEncode_Y.append(encoded)
    else:
        toEncode_CbCr.append(encoded)
    
    

    
def reconstruct_blocks(decodedY, decodedCbCr,index, reconstructed_frame):
    if index[2] == 0:
        channel_type = 'lum'
    else:
        channel_type = 'chr'
    
    assert channel_type in ('lum', 'chr')
        
    if channel_type == 'lum':
        decoded = decodedY[0]
        decodedY.pop(0)
    else:
        decoded = decodedCbCr[0]
        decodedCbCr.pop(0)
        
    decoded = rle.backward(decoded)
    # Reverse RLE + zigzag scanning
    decoded = zigzagScanning.backward(decoded)
    # Dequantization
    decoded = quantization.backward(decoded, channel_type,QP)
    
    # Reverse DCT
    reconstructed_block = dct2d.backward(decoded)
    
    reconstructed_frame.append(reconstructed_block)

In [89]:
toEncode_Y = []
toEncode_CbCr = []
for i in range(0,int(blocks.size/64)-1):
  process_block(blocks[i] ,indices[i] ,toEncode_Y ,toEncode_CbCr)
  

In [90]:
encodedY, codebookY = entropy.huffman_encoding(toEncode_Y)
encodedCbCr, codebookCbCr= entropy.huffman_encoding(toEncode_CbCr)


In [91]:
compression_ratio = rgb_img.shape[0]*rgb_img.shape[1]*24/(sum(len(word) for word in encodedY)+sum(len(word) for word in encodedCbCr))
compression_ratio

100.38373102763431

In [92]:
decodedY = entropy.huffman_decoding(encodedY, codebookY)
decodedCbCr = entropy.huffman_decoding(encodedCbCr, codebookCbCr)


In [93]:
reconstructed_frame = []
print(len(decodedCbCr),len(decodedY),len(indices))
for i in range(0,len(indices)-1):
    reconstruct_blocks(decodedY, decodedCbCr,indices[i], reconstructed_frame)

755007 377504 1132512


In [94]:
ycc_img_compressed = image_block.backward(reconstructed_frame, indices)

In [95]:
ycc_img_compressed = (ycc_img_compressed+128).astype('uint8')

In [96]:
rgb_img_compressed = cv2.cvtColor(ycc_img_compressed, cv2.COLOR_YCrCb2RGB)

In [97]:
Image.fromarray(rgb_img_compressed).save(os.path.join('images', 'result3.jpeg'))